In [1]:
import sys
import os
import importlib.util

def import_all_components():
    current_dir = os.getcwd()
    
    # Retriever
    retriever_path = os.path.join(current_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Query Interface
    query_path = os.path.join(current_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_path)
    query_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_module)
    
    # Answer Generator
    answer_path = os.path.join(current_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_path)
    answer_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_module)
    
    return (retriever_module.DocumentAwareRetriever, 
            query_module.SimpleQueryInterface,
            answer_module.RAGAnswerGenerator)

DocumentAwareRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_all_components()
print("✅ All components imported!")

VECTOR_INDEX_PATH = "../../04_models/vector_index"
retriever = DocumentAwareRetriever(VECTOR_INDEX_PATH)
query_interface = SimpleQueryInterface(retriever)
answer_generator = RAGAnswerGenerator(query_interface)

✅ All components imported!
✓ TF-IDF retriever loaded successfully
✓ Template-based RAG answer generator initialized


# CLI Demo Interface

**Why we're doing this:** 
Quick command-line testing of the full RAG pipeline

**What we're doing:**

- Loading FAISS index and generator components
- Simple input loop for manual query testing
- Displaying answers with source references

🎯 What's Happening:

The input("\n🔍 Your question: ") function is pausing execution and waiting for you to type something and press Enter.

🚀 How to Use It:

- Look for a blinking cursor or input field below the output
- Type one of the example questions like: Tell me about automotive startups in AI
- Press Enter to run the query
- It will process and show you the results
- Then it will ask for another question until you type quit

In [2]:
def working_cli_demo():
    print("\n" + "="*60)
    print("🚗 AUTOMOTIVE TECH RAG - WORKING CLI DEMO")
    print("="*60)
    print("Quick wins to demonstrate:")
    print("1. 'Tell me about automotive startups in AI'")
    print("2. 'What are the challenges in autonomous driving?'") 
    print("3. 'How is AI transforming the automotive industry?'")
    print("Type 'quit' to exit\n")
    
    while True:
        question = input("\n🔍 Your question: ").strip()
        if question.lower() in ['quit', 'exit', 'q']:
            break
            
        print("🔄 Searching through 19,038 document chunks...")
        result = answer_generator.generate_answer(question, top_k=4)
        
        print(f"\n🤖 ANSWER:")
        print(result['answer'])
        
        print(f"\n📚 SOURCES ({len(result['sources'])} found):")
        for i, source in enumerate(result['sources'][:3], 1):
            emoji = "🚀" if source['doc_type'] == 'startups_data' else "📄" if source['doc_type'] == 'research_paper' else "🔍"
            print(f"  {i}. {emoji} {source['source_file']}")
            print(f"     Score: {source['relevance_score']:.3f}")
            print(f"     Preview: {source['content_snippet'][:80]}...")

print("✅ CLI Demo ready - run 'working_cli_demo()' to test")

✅ CLI Demo ready - run 'working_cli_demo()' to test


In [3]:
working_cli_demo()


🚗 AUTOMOTIVE TECH RAG - WORKING CLI DEMO
Quick wins to demonstrate:
1. 'Tell me about automotive startups in AI'
2. 'What are the challenges in autonomous driving?'
3. 'How is AI transforming the automotive industry?'
Type 'quit' to exit

🔄 Searching through 19,038 document chunks...

🤖 ANSWER:
Based on automotive tech_report, startups_data data, here's information about 'What are the latest autimotive trends?':

From tech report:
  • clean-energy output while minimizing 
downtime and operational costs Immersive reality
Technicians u...
  • equity investment, and talent that underpin 
each of the 13 trends and explore the underlying 
techn...

From startups data:
  • STARTUP #17634:
  Name: TrainEarly
  Location: N/A
  Tagline: Healthcare Education Software
  Descri...

Total relevant chunks found: 4

📚 SOURCES (4 found):
  1. 🚀 startups_processed.txt
     Score: 0.448
     Preview: STARTUP #17634:
  Name: TrainEarly
  Location: N/A
  Tagline: Healthcare Educati...
  2. 🔍 mckinsey_tec

# Validation Script

**Why we're doing this:**
Automated testing of retrieval quality across our query set

**What we're doing:**

- Running predefined user queries through the system
- Measuring retrieval relevance scores - to-do
- Logging performance metrics - to-do


In [ ]:
def run_validation():
    print("\n" + "="*60)
    print("🔧 VALIDATION SCRIPT - Testing All Components")
    print("="*60)
    
    test_cases = [
        ("Startups", "Tell me about automotive startups in AI", ["startups_data"]),
        ("Research", "What are the challenges in autonomous driving?", ["research_paper"]),
        ("Mixed", "How is AI transforming the automotive industry?", ["startups_data", "research_paper", "tech_report"])
    ]
    
    all_passed = True
    for name, query, expected_types in test_cases:
        print(f"\n🧪 {name}: '{query}'")
        result = answer_generator.generate_answer(query, top_k=3)
        
        if len(result['sources']) > 0:
            found_types = result['context_used']['document_types_used']
            print(f"   ✅ PASS - {len(result['sources'])} sources, types: {found_types}")
        else:
            print(f"   ❌ FAIL - No sources found")
            all_passed = False
    
    print(f"\n🎯 VALIDATION: {'ALL TESTS PASSED' if all_passed else 'SOME TESTS FAILED'}")
    return all_passed

print("✅ Validation script ready - run 'run_validation()' to test")

✅ Validation script ready - run 'run_validation()' to test


In [6]:
run_validation()


🔧 VALIDATION SCRIPT - Testing All Components

🧪 Startups: 'Tell me about automotive startups in AI'
   ✅ PASS - 3 sources, types: ['research_paper', 'tech_report']

🧪 Research: 'What are the challenges in autonomous driving?'
   ✅ PASS - 3 sources, types: ['research_paper']

🧪 Mixed: 'How is AI transforming the automotive industry?'
   ✅ PASS - 3 sources, types: ['research_paper']

🎯 VALIDATION: ALL TESTS PASSED


True

# Streamline Generation Optimization

**Why we're doing this:** 
Improve response speed and quality for the demo

**What we're doing:**

- Create Streamlit app - done
- Optimizing prompt templates for faster generation - to-do
- Configuring LLM parameters (temperature, max_tokens) - to-do
- Caching frequent retrievals - to-do

In [ ]:

def create_streamlit_app():
    streamlit_code = '''import streamlit as st
import sys
import os
sys.path.append('../03_notebooks')

# Use the same nuclear import for Streamlit
import importlib.util

def import_rag_components():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    parent_dir = os.path.join(current_dir, '..', '03_notebooks')
    
    # Import retriever
    retriever_path = os.path.join(parent_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Import query_interface  
    query_interface_path = os.path.join(parent_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_interface_path)
    query_interface_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_interface_module)
    
    # Import answer_generator
    answer_generator_path = os.path.join(parent_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_generator_path)
    answer_generator_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_generator_module)
    
    return (retriever_module.DocumentAwareRetriever, 
            query_interface_module.SimpleQueryInterface,
            answer_generator_module.RAGAnswerGenerator)

@st.cache_resource
def load_pipeline():
    DocumentAwareRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_rag_components()
    retriever = DocumentAwareRetriever("../04_models/vector_index")
    query_interface = SimpleQueryInterface(retriever)
    return RAGAnswerGenerator(query_interface)

st.set_page_config(page_title="Automotive Tech RAG", page_icon="🚗")
st.title("🚗 Automotive Tech RAG System")
st.markdown("Ask questions about automotive technology, startups, and research")

question = st.text_input("Enter your question:")

if st.button("Search") and question:
    answer_generator = load_pipeline()
    result = answer_generator.generate_answer(question, top_k=5)
    
    st.subheader("Answer")
    st.write(result["answer"])
    
    st.subheader(f"Sources ({len(result['sources'])} found)")
    for source in result["sources"]:
        with st.expander(f"{source['source_file']} (Score: {source['relevance_score']:.3f})"):
            st.write(f"**Type:** {source['doc_type']}")
            st.write(f"**Content:** {source['content_snippet']}")
'''
    
    with open("../05_app/streamlit_app.py", "w") as f:
        f.write(streamlit_code)
    print("✅ Streamlit app created: ../05_app/streamlit_app.py")

create_streamlit_app()

✅ Streamlit app created: ../05_app/streamlit_app.py


**Basic Documentation**

- what is it? Why we doing it?

In [17]:
# TO-DO 4: BASIC DOCUMENTATION
def create_documentation():
    doc_content = '''# Automotive Tech RAG Pipeline

## Quick Start
```python
# Use nuclear import for reliability
from rag_components.retriever import DocumentAwareRetriever
from rag_components.answer_generator import RAGAnswerGenerator

retriever = DocumentAwareRetriever("../04_models/vector_index")
answer_generator = RAGAnswerGenerator(retriever)
result = answer_generator.generate_answer("your question")'''

# End-to-End Integration Test

**Why we're doing this:**
Verify the complete system works before demo

**What we're doing:**

- Testing all document types (papers, reports, patents) - done
- Checking error handling for edge cases - done
- Validating source attribution accuracy - to-do


In [14]:
# DEFINE THE INTEGRATION TEST FUNCTION
def integration_test():
    print("\n" + "="*60)
    print("🔬 END-TO-END INTEGRATION TEST")
    print("="*60)
    
    # Test the 3 quick wins
    quick_wins = [
        ("🚀 Startups", "Tell me about automotive startups in AI"),
        ("📄 Research", "What are the challenges in autonomous driving?"),
        ("🔍 Mixed", "How is AI transforming the automotive industry?")
    ]
    
    all_passed = True
    for name, query in quick_wins:
        print(f"\n{name}: '{query}'")
        result = answer_generator.generate_answer(query, top_k=3)
        
        if len(result['sources']) > 0:
            print(f"   ✅ SUCCESS - {len(result['sources'])} sources")
            print(f"   📄 Document types: {result['context_used']['document_types_used']}")
        else:
            print(f"   ❌ FAILED - No sources found")
            all_passed = False
    
    print(f"\n🎯 INTEGRATION TEST: {'ALL TESTS PASSED' if all_passed else 'SOME TESTS FAILED'}")
    return all_passed

print("✅ Integration test function defined - now you can run it!")

✅ Integration test function defined - now you can run it!


# Complete all Deliverables

Run run_validation() - Ensure all components work
Run create_streamlit_app() - Generate web interface
Run create_documentation() - Create documentation
Run integration_test() - Verify quick wins work

Celebrate! 🎉 RAG pipeline completed.

In [10]:
# Run the validation to ensure everything works
validation_passed = run_validation()


🔧 VALIDATION SCRIPT - Testing All Components

🧪 Startups: 'Tell me about automotive startups in AI'
   ✅ PASS - 3 sources, types: ['research_paper', 'tech_report']

🧪 Research: 'What are the challenges in autonomous driving?'
   ✅ PASS - 3 sources, types: ['research_paper']

🧪 Mixed: 'How is AI transforming the automotive industry?'
   ✅ PASS - 3 sources, types: ['research_paper']

🎯 VALIDATION: ALL TESTS PASSED


In [11]:
# This creates the web interface
create_streamlit_app()

✅ Streamlit app created: ../05_app/streamlit_app.py


In [12]:
# This generates the basic documentation
create_documentation()

In [15]:
# This automatically tests the 3 quick wins
integration_test()


🔬 END-TO-END INTEGRATION TEST

🚀 Startups: 'Tell me about automotive startups in AI'
   ✅ SUCCESS - 3 sources
   📄 Document types: ['research_paper', 'tech_report']

📄 Research: 'What are the challenges in autonomous driving?'
   ✅ SUCCESS - 3 sources
   📄 Document types: ['research_paper']

🔍 Mixed: 'How is AI transforming the automotive industry?'
   ✅ SUCCESS - 3 sources
   📄 Document types: ['research_paper']

🎯 INTEGRATION TEST: ALL TESTS PASSED


True